In [8]:
import sys
print(sys.executable)

TRAINING_DATA_PATH = "/Users/slouvan/sandbox/anago/data/conll2003/en/ner/train.lower.txt"
DEV_DATA_PATH = "/Users/slouvan/sandbox/anago/data/conll2003/en/ner/valid.lower.txt"
TEST_DATA_PATH = "/Users/slouvan/sandbox/anago/data/conll2003/en/ner/test.lower.txt"


import anago
from anago.reader import load_data_and_labels, load_glove


x_train, y_train = load_data_and_labels(TRAINING_DATA_PATH)
x_dev, y_dev = load_data_and_labels(DEV_DATA_PATH)
x_test, y_test = load_data_and_labels(TEST_DATA_PATH)
print(len(x_dev))
print(len(y_test))
#embeddings = load_glove("/Users/slouvan/sandbox/cross-domain/data/glove.6B.100d.txt")


#model = anago.Sequence(char_feature=True)
#model.train(x_train[:200], y_train[:200], x_dev[:200], y_dev[:200])

#model = model.load("/Users/slouvan/sandbox/anago/checkpoints/models/WE_CE_FIXED_02_05_2/")

def __startOfChunk(prevTag, tag, prevTagType, tagType, chunkStart=False):
    if prevTag == 'B' and tag == 'B':
        chunkStart = True
    if prevTag == 'I' and tag == 'B':
        chunkStart = True
    if prevTag == 'O' and tag == 'B':
        chunkStart = True
    if prevTag == 'O' and tag == 'I':
        chunkStart = True

    if prevTag == 'E' and tag == 'E':
        chunkStart = True
    if prevTag == 'E' and tag == 'I':
        chunkStart = True
    if prevTag == 'O' and tag == 'E':
        chunkStart = True
    if prevTag == 'O' and tag == 'I':
        chunkStart = True

    if tag != 'O' and tag != '.' and prevTagType != tagType:
        chunkStart = True
    return chunkStart

def __endOfChunk(prevTag, tag, prevTagType, tagType, chunkEnd=False):
    if prevTag == 'B' and tag == 'B':
        chunkEnd = True
    if prevTag == 'B' and tag == 'O':
        chunkEnd = True
    if prevTag == 'I' and tag == 'B':
        chunkEnd = True
    if prevTag == 'I' and tag == 'O':
        chunkEnd = True

    if prevTag == 'E' and tag == 'E':
        chunkEnd = True
    if prevTag == 'E' and tag == 'I':
        chunkEnd = True
    if prevTag == 'E' and tag == 'O':
        chunkEnd = True
    if prevTag == 'I' and tag == 'O':
        chunkEnd = True

    if prevTag != 'O' and prevTag != '.' and prevTagType != tagType:
        chunkEnd = True
    return chunkEnd

def __splitTagType(tag):
    s = tag.split('-')
    if len(s) > 2 or len(s) == 0:
        raise ValueError('tag format wrong. it must be B-xxx.xxx')
    if len(s) == 1:
        tag = s[0]
        tagType = ""
    else:
        tag = s[0]
        tagType = s[1]
    return tag, tagType

def computeF1ScoreCONLL(correct_slots, pred_slots):
    correctChunk = {}
    correctChunkCnt = 0
    foundCorrect = {}
    foundCorrectCnt = 0
    foundPred = {}
    foundPredCnt = 0
    correctTags = 0
    tokenCount = 0
    for correct_slot, pred_slot in zip(correct_slots, pred_slots):
        inCorrect = False
        lastCorrectTag = 'O'
        lastCorrectType = ''
        lastPredTag = 'O'
        lastPredType = ''
        for c, p in zip(correct_slot, pred_slot):
            correctTag, correctType = __splitTagType(c)
            predTag, predType = __splitTagType(p)

            if inCorrect == True:
                if __endOfChunk(lastCorrectTag, correctTag, lastCorrectType, correctType) == True and \
                        __endOfChunk(lastPredTag, predTag, lastPredType, predType) == True and \
                        (lastCorrectType == lastPredType):
                    inCorrect = False
                    correctChunkCnt += 1
                    if lastCorrectType in correctChunk:
                        correctChunk[lastCorrectType] += 1
                    else:
                        correctChunk[lastCorrectType] = 1
                elif __endOfChunk(lastCorrectTag, correctTag, lastCorrectType, correctType) != \
                        __endOfChunk(lastPredTag, predTag, lastPredType, predType) or \
                        (correctType != predType):
                    inCorrect = False

            if __startOfChunk(lastCorrectTag, correctTag, lastCorrectType, correctType) == True and \
                    __startOfChunk(lastPredTag, predTag, lastPredType, predType) == True and \
                    (correctType == predType):
                inCorrect = True

            if __startOfChunk(lastCorrectTag, correctTag, lastCorrectType, correctType) == True:
                foundCorrectCnt += 1
                if correctType in foundCorrect:
                    foundCorrect[correctType] += 1
                else:
                    foundCorrect[correctType] = 1

            if __startOfChunk(lastPredTag, predTag, lastPredType, predType) == True:
                foundPredCnt += 1
                if predType in foundPred:
                    foundPred[predType] += 1
                else:
                    foundPred[predType] = 1

            if correctTag == predTag and correctType == predType:
                correctTags += 1

            tokenCount += 1

            lastCorrectTag = correctTag
            lastCorrectType = correctType
            lastPredTag = predTag
            lastPredType = predType

        if inCorrect == True:
            correctChunkCnt += 1
            if lastCorrectType in correctChunk:
                correctChunk[lastCorrectType] += 1
            else:
                correctChunk[lastCorrectType] = 1
    print("foundPredCnt : {}   correctChunkCnt : {}   ".format(foundPredCnt, correctChunkCnt))

    if foundPredCnt > 0:
        precision = 100 * correctChunkCnt / foundPredCnt
    else:
        precision = 0

    if foundCorrectCnt > 0:
        recall = 100 * correctChunkCnt / foundCorrectCnt
    else:
        recall = 0

    if (precision + recall) > 0:
        f1 = (2 * precision * recall) / (precision + recall)
    else:
        f1 = 0

    return f1, precision, recall

/Users/slouvan/anaconda3/envs/anago36/bin/python
3250
3453


In [6]:
x_test = [x for x in x_test if len(x) > 1]
y_test = [y for y in y_test if len(y) > 1]
print(type(x_test))
#print(x_test[0])
#print(y_test[0])
#for idx in range(len(x_test)):
#    model.eval([x_test[idx]], [y_test[idx]])
#model.eval(x_test, y_test)x
# compute f1 score is modified from conlleval.pl
#model.model.summary()
#from keras.utils import plot_model
#plot_model(model.model, to_file='model.png')

<type 'list'>
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 1   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 1   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating


Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 


Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 1   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 


Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 3   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 1   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 2   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 


Evaluating F1 score of 1 number of predictions 
foundPredCnt : 1   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 


...

Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 


Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 1   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 


Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 


Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 


Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 1   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 
foundPredCnt : 0   correctChunkCnt : 0   
 - f1: 0.00
F1score is constructed
Evaluating
Evaluating F1 score of 1 number of predictions 


In [7]:
y_pred = []
with open("/Users/slouvan/sandbox/anago/data/conll2003/en/ner/test.prediction.lower.txt", "w") as f:
    for id_sent, sent in enumerate(x_test):
        predictions = model.analyze(sent)
        y_pred.append(predictions[1])
        for id_token, token in enumerate(sent):
            #print("{} {} {}".format(sent[id_token], y_train[id_sent][id_token], predictions[1][id_token]))
            f.write(sent[id_token]+" "+y_test[id_sent][id_token]+" "+predictions[1][id_token]+"\n")
        f.write("\n")

[array([[ 607, 2012,  198, 1115,    1, 2270,  203,   75,  530,    1,    1,
         488]]), array([[[ 2,  4,  9,  9, 12, 11,  0,  0],
        [17,  0,  0,  0,  0,  0,  0,  0],
        [26,  6, 13,  6, 14,  0,  0,  0],
        [ 5, 12, 15,  0,  0,  0,  0,  0],
        [ 3,  7,  9, 25, 18,  0,  0,  0],
        [28,  8, 14,  0,  0,  0,  0,  0],
        [33,  0,  0,  0,  0,  0,  0,  0],
        [ 9, 19,  8, 14,  6,  0,  0,  0],
        [ 8, 14,  0,  0,  0,  0,  0,  0],
        [ 2,  7, 11, 13, 11,  8,  2, 12],
        [10, 12, 16, 12,  6, 15,  0,  0],
        [24,  0,  0,  0,  0,  0,  0,  0]]])]
[array([[1, 1]]), array([[[14,  6, 10,  8, 20],
        [ 3,  6, 10, 25,  8]]])]
[array([[   1,  203, 2328,  590, 1782, 1086]]), array([[[ 6,  3, 17,  6,  8, 14,  0,  0,  0,  0],
        [33,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 7, 14,  8, 15, 12, 10,  0,  0,  0,  0],
        [ 6, 11,  6, 31,  0,  0,  0,  0,  0,  0],
        [12, 20,  8, 11,  6, 15, 12,  2,  0,  0],
        [ 1,  1,  1,  

[array([[   1,  147, 1806,  248, 1480, 2440, 1897, 1062, 1350, 2710, 1350,
           1,  530,  926, 1625,  938,  488]]), array([[[19,  4,  2, 15,  2,  0,  0,  0,  0],
        [ 7,  6, 12,  0,  0,  0,  0,  0,  0],
        [13,  3,  6, 18,  0,  0,  0,  0,  0],
        [25,  7, 28,  6,  8, 15,  0,  0,  0],
        [ 6, 14, 10,  0,  0,  0,  0,  0,  0],
        [ 2,  4,  7, 15, 19,  0,  0,  0,  0],
        [25,  4, 11, 12,  6,  0,  0,  0,  0],
        [15,  6, 25, 12,  0,  0,  0,  0,  0],
        [ 4, 14,  0,  0,  0,  0,  0,  0,  0],
        [ 8, 14, 10,  4, 14, 12,  2,  8,  6],
        [ 4, 14,  0,  0,  0,  0,  0,  0,  0],
        [ 2,  6, 15,  7, 11, 10,  6, 18,  0],
        [ 8, 14,  0,  0,  0,  0,  0,  0,  0],
        [ 5, 11,  4,  7, 13,  0,  0,  0,  0],
        [ 6,  0,  0,  0,  0,  0,  0,  0,  0],
        [20,  6, 15,  9, 19, 12,  2,  0,  0],
        [24,  0,  0,  0,  0,  0,  0,  0,  0]]])]
[array([[1448,    7, 2794, 1885,  770, 1395, 1007,    1,  230, 2282, 1007,
        2037,  488

[array([[ 525,  672,  562,  311, 1418,  641,    1,  530,    7, 2855,  488]]), array([[[ 8, 15,  0,  0,  0,  0,  0,  0,  0],
        [28,  6,  2,  0,  0,  0,  0,  0,  0],
        [15, 19, 12,  0,  0,  0,  0,  0,  0],
        [ 2, 12,  9,  4, 14, 10,  0,  0,  0],
        [ 2, 18, 11,  8,  6, 14,  0,  0,  0],
        [10, 12, 16, 12, 14,  2,  8, 27, 12],
        [31,  3,  7, 14, 10, 12, 11,  0,  0],
        [ 8, 14,  0,  0,  0,  0,  0,  0,  0],
        [16,  4,  7, 11,  0,  0,  0,  0,  0],
        [20,  8, 14,  7, 15, 12,  2,  0,  0],
        [24,  0,  0,  0,  0,  0,  0,  0,  0]]])]
[array([[   1,  917,    1, 1286, 2741,    1, 1625, 1008,    1, 2095,  562,
           1,  530,  562,  314,  769, 1725, 1107,    1, 2741,    1,  525,
        2095,  562,  290,    1, 2312,    1,  716, 1604,  488]]), array([[[10, 12, 16, 12, 14, 10, 12, 11,  0],
        [19,  6,  2,  2,  6, 14,  0,  0,  0],
        [ 6, 31, 31,  6,  2,  0,  0,  0,  0],
        [11,  4,  2, 12,  0,  0,  0,  0,  0],
        [15,  4

[array([[1044,    1,    1, 2543, 2098, 2688,  883]]), array([[[ 1, 24,  0,  0,  0,  0,  0],
        [ 9,  6, 14, 10,  8,  9, 12],
        [ 5,  8,  3,  5,  0,  0,  0],
        [42,  0,  0,  0,  0,  0,  0],
        [16, 11,  6, 14,  9, 12,  0],
        [43,  0,  0,  0,  0,  0,  0],
        [ 1,  1, 24,  1,  1,  0,  0]]])]
[array([[1044,    1,    1, 2543, 1724, 2688,  883]]), array([[[ 1, 24,  0,  0,  0,  0,  0],
        [20,  8, 14, 14,  6,  0,  0],
        [25,  6, 11, 19,  7,  0,  0],
        [42,  0,  0,  0,  0,  0,  0],
        [16,  8, 14,  3,  6, 14, 10],
        [43,  0,  0,  0,  0,  0,  0],
        [ 1,  1, 24, 22,  1,  0,  0]]])]
[array([[1044,    1,    1, 2543,  198, 2688,  883]]), array([[[ 1, 24,  0,  0,  0,  0],
        [15,  6, 12,  0,  0,  0],
        [ 2,  6, 15,  4, 18,  6],
        [42,  0,  0,  0,  0,  0],
        [26,  6, 13,  6, 14,  0],
        [43,  0,  0,  0,  0,  0],
        [ 1,  1, 24,  1,  1,  0]]])]
[array([[1044,    1,    1, 2543,  260, 2688,  883]]), array

[array([[ 612, 2125,  944, 2411,  680]]), array([[[ 2,  6,  3,  8, 20],
        [20,  6,  3,  8, 25],
        [14,  4, 15,  0,  0],
        [ 4,  7, 15,  0,  0],
        [ 1,  0,  0,  0,  0]]])]
[array([[2893, 2543,    1, 1593,    1, 2688,   97]]), array([[[12, 21, 15, 11,  6,  2],
        [42,  0,  0,  0,  0,  0],
        [ 3, 31, 17,  1,  0,  0],
        [14, 31, 17,  1,  0,  0],
        [28, 17,  1,  1,  0,  0],
        [43,  0,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0]]])]
[array([[ 104, 2543,  628,  680, 2160,  530,   97,    1, 2688,   50]]), array([[[15,  4, 15,  6,  3,  0,  0],
        [42,  0,  0,  0,  0,  0,  0],
        [16,  4, 11,  0,  0,  0,  0],
        [ 1,  0,  0,  0,  0,  0,  0],
        [28,  8,  9, 25, 12, 15,  2],
        [ 8, 14,  0,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0,  0],
        [ 4, 27, 12, 11,  2,  0,  0],
        [43,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  1,  0,  0,  0,  0]]])]
[array([[1479, 2312,    1, 2109,  828, 2543,    1, 2282

...

[array([[   1,  680,   97,  244,   97, 1400]]), array([[[10, 12, 14, 27, 12, 11],
        [ 1,  0,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0],
        [24,  1,  1,  1,  0,  0],
        [ 1,  1,  0,  0,  0,  0],
        [ 1, 23,  1,  0,  0,  0]]])]
[array([[1047, 2713,  680,   97,  244,   97, 1400]]), array([[[ 2,  6, 14,  0,  0,  0,  0],
        [ 6, 14, 15,  4, 14,  8,  4],
        [ 1,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0,  0],
        [24,  1,  1,  1,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0,  0],
        [ 1, 23,  1,  0,  0,  0,  0]]])]
[array([[   1,  680,   97,  244,   97, 1400]]), array([[[27,  6, 14,  9,  4,  7, 27, 12, 11],
        [ 1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0,  0,  0,  0],
        [24,  1,  1,  1,  0,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 23,  1,  0,  0,  0,  0,  0,  0]]])]
[array([[   1, 1595]]), array([[[13,  6,  9,  8, 16,  8,  9,  0],
        [10,  8, 27,  8,  2, 

[array([[321,  97,  97, 680,  97,  97,  97]]), array([[[31,  4,  2, 15,  4, 14],
        [ 1, 22,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0],
        [ 1,  0,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0]]])]
[array([[1029,   97,   97,  680,   97,   97,   97]]), array([[[13,  8, 15, 15,  2, 31,  7, 11,  5, 19],
        [ 1, 22,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0,  0,  0,  0,  0]]])]
[array([[  1, 680,  97, 680,  97,  97,  97]]), array([[[ 4, 15, 15,  6, 28,  6],
        [ 1,  0,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0],
        [ 1,  0,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0],
        [ 1, 22,  0,  0,  0,  0]]])]
[array([[   1, 15

[array([[   1,    1,    1, 2012, 1711,  716, 1477,  488]]), array([[[14, 19,  3,  0,  0,  0,  0],
        [ 8,  9, 12,  0,  0,  0,  0],
        [19,  4,  9, 25, 12, 18,  0],
        [17,  0,  0,  0,  0,  0,  0],
        [16, 11,  8, 10,  6, 18,  0],
        [35,  2,  0,  0,  0,  0,  0],
        [11, 12,  2,  7,  3, 15,  2],
        [24,  0,  0,  0,  0,  0,  0]]])]
[array([[  63, 2425, 1086]]), array([[[14, 12, 28,  0,  0,  0,  0,  0,  0,  0],
        [18,  4, 11, 25,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  1,  1, 17,  1,  1, 17, 22,  1]]])]
[array([[1477, 2312, 2816,    1]]), array([[[11, 12,  2,  7,  3, 15,  2,  0],
        [ 4, 16,  0,  0,  0,  0,  0,  0],
        [14,  6, 15,  8,  4, 14,  6,  3],
        [19,  4,  9, 25, 12, 18,  0,  0]]])]
[array([[1832, 2695, 1350, 1711, 2543, 2776,  776,  530,  589, 2688, 2109]]), array([[[ 3, 12,  6,  5,  7, 12],
        [ 5,  6, 20, 12,  2,  0],
        [ 4, 14,  0,  0,  0,  0],
        [16, 11,  8, 10,  6, 18],
        [42,  0,  0,  0,  0, 

[array([[ 562,  424,    1,    1, 1350,   97, 2625,  165,   97, 2695,  203,
           7, 1536, 2312,    1,  488]]), array([[[15, 19, 12,  0,  0,  0,  0,  0,  0],
        [11, 12,  2,  7,  3, 15,  0,  0,  0],
        [ 3, 12,  6, 27, 12,  2,  0,  0,  0],
        [11, 12,  6,  3,  0,  0,  0,  0,  0],
        [ 4, 14,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0,  0,  0,  0],
        [13,  4,  8, 14, 15,  2,  0,  0,  0],
        [ 6, 16, 15, 12, 11,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0,  0,  0,  0],
        [ 5,  6, 20, 12,  2,  0,  0,  0,  0],
        [33,  0,  0,  0,  0,  0,  0,  0,  0],
        [16,  4,  7, 11,  0,  0,  0,  0,  0],
        [ 6, 19, 12,  6, 10,  0,  0,  0,  0],
        [ 4, 16,  0,  0,  0,  0,  0,  0,  0],
        [31,  6, 11,  9, 12,  3,  4, 14,  6],
        [24,  0,  0,  0,  0,  0,  0,  0,  0]]])]
[array([[1395, 1723, 1007, 1832, 1961, 1965,  200,  562,   63,  966,    1,
         203,    1, 1885, 1802,    1, 2312,    1,    1, 1536, 2312, 24

[array([[1515,    1,   97,  680,  680,  680,   97,   97,   97]]), array([[[11,  6,  9,  8, 14,  5,  0,  0,  0],
        [ 2,  6, 14, 15,  6, 14, 10, 12, 11],
        [ 1,  1,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0,  0,  0,  0]]])]
[array([[  1,   1,  97, 680, 680, 680,  97,  97,  97]]), array([[[11,  6, 18,  4,  0,  0,  0,  0,  0],
        [27,  6,  3,  3, 12,  9,  6, 14,  4],
        [ 1,  1,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 22,  0,  0,  0,  0,  0,  0,  0]]])]
[array([[  1,  97, 680, 680, 680,  97,  97,

In [8]:
f1, p, r = computeF1ScoreCONLL(y_test, y_pred)

foundPredCnt : 4961   correctChunkCnt : 3944   


In [9]:
print(f1)

74.42211529389564


1.5603900975243812


In [28]:
label_conll = []
label_eval = []
sent_cnt = 0
with open("/Users/slouvan/sandbox/anago/data/conll2003/en/ner/test.prediction.lower.txt", "r") as f:
    for line in f:
        fields = line.split(" ")
        if len(fields) > 1:
            label_conll.append(fields[2])
        else :
            label_conll.append(fields[0])
            

with open("/Users/slouvan/sandbox/anago/data/conll2003/en/ner/test.prediction.eval.txt", "r") as f:
    for line in f:
        fields = line.split("\t")
        if len(fields) > 1:
            label_eval.append(fields[2])
        else :
            label_eval.append(fields[0])

In [29]:
print(len(label_conll))
print(len(label_eval))

cnt = 0
for idx in range(len(label_conll)):
    #print(idx)
    #print("{} {}".format(type(label_conll[idx]), type(label_eval[idx])))
    #print(label_conll[idx])
    if label_conll[idx].strip() != label_eval[idx].strip():
        print("WEIRD idx {} \t {} {}".format(idx + 1, label_conll[idx].strip(), label_eval[idx].strip()))
        cnt += 1
print(cnt)


49822
49822
WEIRD idx 14 	 B-PER O
WEIRD idx 15 	 I-PER O
WEIRD idx 95 	 I-PER O
WEIRD idx 210 	 B-PER O
WEIRD idx 232 	 B-PER O
WEIRD idx 288 	 B-ORG O
WEIRD idx 441 	 I-ORG O
WEIRD idx 663 	 I-PER O
WEIRD idx 703 	 I-PER O
WEIRD idx 748 	 B-LOC O
WEIRD idx 762 	 B-PER O
WEIRD idx 833 	 B-ORG O
WEIRD idx 854 	 B-PER O
WEIRD idx 871 	 B-ORG O
WEIRD idx 872 	 I-ORG O
WEIRD idx 925 	 B-PER O
WEIRD idx 1470 	 B-PER B-LOC
WEIRD idx 1524 	 B-ORG I-ORG
WEIRD idx 1553 	 B-ORG O
WEIRD idx 1718 	 I-PER O
WEIRD idx 1898 	 B-PER O
WEIRD idx 2010 	 B-ORG O
WEIRD idx 2304 	 B-ORG I-ORG
WEIRD idx 2323 	 B-LOC I-PER
WEIRD idx 2452 	 B-ORG I-ORG
WEIRD idx 2546 	 B-PER O
WEIRD idx 2657 	 B-PER O
WEIRD idx 2692 	 I-LOC O
WEIRD idx 2801 	 I-PER O
WEIRD idx 2807 	 B-PER O
WEIRD idx 3036 	 B-PER O
WEIRD idx 3069 	 I-PER O
WEIRD idx 3103 	 B-PER O
WEIRD idx 3374 	 B-PER O
WEIRD idx 3375 	 I-PER O
WEIRD idx 3395 	 B-PER O
WEIRD idx 3482 	 I-PER O
WEIRD idx 4029 	 I-ORG O
WEIRD idx 4036 	 I-PER O
WEIRD idx 41

In [19]:
model = model.load("/Users/slouvan/sandbox/anago/checkpoints/models/WE_CE_FIXED/")
model.analyze("nadim ladki".split())

PREPROCESSOR FILE : /Users/slouvan/sandbox/anago/checkpoints/models/WE_CE_FIXED/preprocessor.pkl
THE VOCABULARY SIZE 1 IS 2898
[array([[1, 1]]), array([[[ 3,  2, 12, 14,  8],
        [24,  2, 12, 23, 14]]])]


({'words': ['nadim', 'ladki'],
  'entities': [{'text': 'nadim',
    'type': 'PER',
    'score': 1.0,
    'beginOffset': 0,
    'endOffset': 1}]},
 ['B-PER', 'O'],
 array([ 1.,  1.], dtype=float32),
 {'words': ['nadim', 'ladki'],
  'entities': [{'text': 'nadim',
    'type': 'PER',
    'score': 1.0,
    'beginOffset': 0,
    'endOffset': 1}]})